<a href="https://colab.research.google.com/github/abhishek-shishodia/ML_parts/blob/column-transformer/column_transformer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [46]:
import pandas as pd
import numpy as np

In [47]:
df = pd.read_csv('/content/covid_toy.csv')

In [48]:
df.sample(3)

,age,gender,fever,cough,city,has_covid
13,64,Male,102.0,Mild,Bangalore,Yes
80,14,Female,99.0,Mild,Mumbai,Yes
90,59,Female,99.0,Strong,Delhi,No


In [49]:
from sklearn.model_selection import train_test_split
Xtrain,Xtest,Ytrain,Ytest = train_test_split(df.drop(columns= ['has_covid']),df['has_covid'],test_size=0.3,random_state=0)

In [50]:
Xtrain.shape

(70, 5)

In [51]:
# old life
# simple imputer
from sklearn.impute import SimpleImputer
si = SimpleImputer()
s_train = si.fit_transform(Xtrain[['fever']])
s_test = si.fit_transform(Xtest[['fever']])

In [52]:
# onehotencoder
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder
ohe = OneHotEncoder(drop = 'first',sparse =False)
xgc = ohe.fit_transform(Xtrain[['gender','city']])
xtgc = ohe.fit_transform(Xtest[['gender','city']])

# ordinalencoder
oe = OrdinalEncoder(categories=[['Mild','Strong']])
oetrain = oe.fit_transform(Xtrain[['cough']])
oetest = oe.fit_transform(Xtest[['cough']])


/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [53]:
# Extracting Age
X_train_age = Xtrain.drop(columns=['gender','fever','cough','city']).values

# also the test data
X_test_age = Xtest.drop(columns=['gender','fever','cough','city']).values

X_train_age.shape

(70, 1)

In [54]:
X_train_transformed = np.concatenate((X_train_age,s_train,xgc,oetrain),axis=1)
# also the test data
X_test_transformed = np.concatenate((X_test_age,s_test,xtgc,oetest),axis=1)

X_train_transformed.shape

(70, 7)

COLUMN TRANSFORMER

In [55]:
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OrdinalEncoder, OneHotEncoder

transformer = ColumnTransformer(transformers=[
    ('tnf1', SimpleImputer(), ['fever']),
    ('tnf2', OrdinalEncoder(categories=[['Mild', 'Strong']]), ['cough']),
    ('tnf3', OneHotEncoder(sparse=False, drop='first'), ['gender', 'city'])
], remainder='passthrough')


In [56]:
transformer.fit_transform(Xtrain).shape

/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


(70, 7)

In [57]:
transformer.fit_transform(Xtest).shape

/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


(30, 7)